# Formel 1 Datenanalyse mit OpenF1 API

In diesem Notebook werden wir die OpenF1 API nutzen, um Formel 1 Daten zu analysieren. Die API bietet Zugriff auf Echtzeit-Renndaten, Rundenzeiten, und mehr.

## Was wir lernen werden:
- API Abfragen mit requests
- Datenverarbeitung mit pandas
- Praktische Datenanalyse an echten F1-Daten

In [1]:
# Benötigte Bibliotheken importieren
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np

In [8]:
API_BASE_URL = "https://api.openf1.org/v1/"

In [25]:
import io

# Fetch sessions for 2023 in CSV format and load into a pandas DataFrame
params = {'year': 2023, 'csv': 'true'}
url = f"{API_BASE_URL}sessions"  # API_BASE_URL is defined in the notebook
print(f"Die URL die abgefragt wird lautet: {url}")

resp = requests.get(url, params=params, timeout=20)
resp.raise_for_status()  # raise an error if the request failed

df_sessions_2023 = pd.read_csv(io.StringIO(resp.text))
df_sessions_2023.head()

Die URL die abgefragt wird lautet: https://api.openf1.org/v1/sessions


HTTPError: 503 Server Error: Service Unavailable for url: https://api.openf1.org/v1/sessions?year=2023&csv=true

In [10]:
# find the latest session in df_sessions_2023 by start time
latest_idx = pd.to_datetime(df_sessions_2023['date_start']).idxmax()
latest_session = df_sessions_2023.loc[latest_idx]
latest_session

circuit_key                                  70
circuit_short_name           Yas Marina Circuit
country_code                                UAE
country_key                                  21
country_name               United Arab Emirates
date_end              2023-11-26 15:00:00+00:00
date_start            2023-11-26 13:00:00+00:00
gmt_offset                             04:00:00
location                             Yas Island
meeting_key                                1226
session_key                                9197
session_name                               Race
session_type                               Race
year                                       2023
Name: 112, dtype: object

In [26]:
from io import StringIO

session_key = 9197  # bekannter, funktionierender Testwert

url = f"{API_BASE_URL}drivers?session_key={session_key}"
print(f"Abfrage: {url}")
resp = requests.get(url, timeout=15)
try:
    resp.raise_for_status()
except Exception as e:
    print(f"HTTP-Fehler: {e}")
    drivers_in_session = pd.DataFrame()
else:
    text = resp.text.strip()
    # JSON-Fall
    if text.startswith('{') or text.startswith('[') or 'application/json' in resp.headers.get('content-type', ''):
        data = resp.json()
        # Prüfe bekannte Wrapper-Felder
        drivers_in_session = None
        if isinstance(data, dict):
            for key in ('data', 'drivers', 'items', 'results'):
                if key in data and isinstance(data[key], list):
                    drivers_in_session = pd.DataFrame(data[key])
                    break
            if drivers_in_session is None:
                # dict als einzelne Zeile versuchen
                try:
                    drivers_in_session = pd.DataFrame([data])
                except Exception:
                    drivers_in_session = pd.DataFrame()
        elif isinstance(data, list):
            drivers_in_session = pd.DataFrame(data)
        else:
            drivers_in_session = pd.DataFrame()
    else:
        # CSV/Text-Fallback
        try:
            drivers_in_session = pd.read_csv(StringIO(resp.text))
        except Exception:
            drivers_in_session = pd.DataFrame()

if drivers_in_session is None or drivers_in_session.empty:
    print(f'Keine Fahrerinformationen gefunden für session_key={session_key}')
else:
    # Kurz zeigen; weitere Bereinigung/Normalisierung optional
    display(drivers_in_session.head(200))

Abfrage: https://api.openf1.org/v1/drivers?session_key=9197
HTTP-Fehler: 504 Server Error: Gateway Timeout for url: https://api.openf1.org/v1/drivers?session_key=9197
Keine Fahrerinformationen gefunden für session_key=9197
HTTP-Fehler: 504 Server Error: Gateway Timeout for url: https://api.openf1.org/v1/drivers?session_key=9197
Keine Fahrerinformationen gefunden für session_key=9197
